# Network eCREST



## Setup

Do the following two setup steps regardless of how you will be using this script. 

### 1. Imports

Run the following code cell to import the necessary packages and modules. 

In [13]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
import random
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new

from eCREST_cli import ecrest, import_settings

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


## Create viewer

In [113]:
viewer = neuroglancer.Viewer()
viewer.set_state({})

location=[17000,17000,1500]

with viewer.config_state.txn() as s:
    s.show_layer_panel = True ###

with viewer.txn(overwrite=True) as s:
           
    dimensions = neuroglancer.CoordinateSpace(
        scales=[16, 16, 30],# self.vx_sizes['em'],
        units='nm',
        names=['x', 'y', 'z']   )

    s.showSlices = False
    s.dimensions = dimensions
    s.position = array(location)
    s.layout = "3d"
    s.projectionScale = 30000
    # s.projection_background_color= "#000000"
    s.projection_background_color= "#ffffff"
    s.showSlices = True
    
with viewer.txn(overwrite=True) as s:
    wb_open(str(viewer))

In [117]:
db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()

a = ', '.join(['base_address'])

db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')

[base_seg] = db_cursors.fetchall()[0]



two_d_intensity = 0.5

with viewer.txn(overwrite=True) as s:

    s.layers['base_segs'] = neuroglancer.SegmentationLayer(source = base_seg, segments=[], segment_colors={})
    s.layers['base_segs'].ignoreNullVisibleSet = False
    s.layers['base_segs'].pick = True
    s.layers['base_segs'].selectedAlpha = two_d_intensity #For 2D

In [115]:
with viewer.txn(overwrite=True) as s:
    s.layers['em'] = neuroglancer.ImageLayer(source = 'brainmaps://10393113184:ell:roi450um_xyz')

### remove displayed segments

In [24]:
displayed_segs = set([str(x) for x in viewer.state.layers['base_segs'].segments])

In [25]:
# REMOVE SEGMENTS FROM NGviewer
remove_segs = displayed_segs

with viewer.txn(overwrite=True) as s:
    for bs in remove_segs:
        if int(bs) in s.layers['base_segs'].segments:
            s.layers['base_segs'].segments.remove(int(bs))

## vizualize post-synaptic annotations

color by whether the segment is part of a current reconstrution or not

In [65]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    

all_base_seg = []

for x in nodes:
    cell = ecrest(settings_dict,filepath = nodefiles[x],launch_viewer=False)
    all_base_seg.extend([a for b in cell.cell_data['base_segments'].values() for a in b])


In [71]:
all_syn = []

for x_pre in nodes:
    pre = ecrest(settings_dict,filepath = nodefiles[x_pre],launch_viewer=False)

    # report if the cell is not cell-typed
    if (pre.get_ctype('manual') == []) | (pre.get_ctype('manual') == ''):
        print(f'cell {x_pre} is not cell-typed in json')
    
    # check if the cell is an mg cell and if it has post-synaptic segments labeled
    if ('mg' in pre.get_ctype('manual')) & (pre.cell_data['end_points']['post-synaptic'] != []):
        try:
            all_syn.extend(pre.cell_data['end_points']['post-synaptic'])
            
        except IndexError as msg:
                # cellid = x_pre['name']
                print(msg, f'for cell {pre.cell_data["metadata"]["main_seg"]["base"]} -- at least one synapse has no segment id')



df = pd.DataFrame(all_syn, columns = ['x','y','z','segid'])

syn_missing = set(set(df['segid'].values)).difference(set(all_base_seg))

syn_found = set(df['segid'].values).difference(syn_missing)

In [116]:
vx_sizes = [16, 16, 30]

point_type = 'reconstructed'
with viewer.txn(overwrite=True) as s:
    s.layers[point_type] = neuroglancer.AnnotationLayer()
    s.layers[point_type].annotationColor = '#ff0000'
    
    for pos,r in df[df['segid'].isin(list(syn_missing))][['x','y','z']].iterrows():
        point = [r[0],r[1],r[2]]
    # for pos, point in enumerate(self.cell_data['end_points'][point_type]):
        point_array = array([int(point[x]/vx_sizes[x]) for x in range(3)])
        point_id = f'{point_type}_{pos}'
        pa = neuroglancer.PointAnnotation(id=point_id, point = point_array)
        s.layers[point_type].annotations.append(pa)    

point_type = 'missing'
with viewer.txn(overwrite=True) as s:
    s.layers[point_type] = neuroglancer.AnnotationLayer()
    s.layers[point_type].annotationColor = '#228b22'

    for pos,r in df[df['segid'].isin(list(syn_found))][['x','y','z']].iterrows():
        point = [r[0],r[1],r[2]]
    # for pos, point in enumerate(self.cell_data['end_points'][point_type]):
        point_array = array([int(point[x]/vx_sizes[x]) for x in range(3)])
        point_id = f'{point_type}_{pos}'
        pa = neuroglancer.PointAnnotation(id=point_id, point = point_array)
        s.layers[point_type].annotations.append(pa)    


## visualize overlapping segments for duplicates

In [ ]:
overlapping_cells = ['474236060','644824967']

In [37]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

nodes = [child.name.split('_')[2] for child in sorted(dirpath.iterdir()) 
         if (child.name[0]!='.') & (child.is_file())] # ignore hidden files]

nodefiles = dict()
for child in sorted(dirpath.iterdir()):
    if (child.name[0]!='.') & (child.is_file()):
        nodefiles[child.name.split('_')[2]] = child
                    
# Create a base_segments dictionary of all cells

base_segments = {}
cell_type={}
for x in [n for n in nodes if n in overlapping_cells]:
    cell = ecrest(settings_dict,filepath = nodefiles[x],launch_viewer=False)
    base_segments[cell.cell_data['metadata']['main_seg']['base']] = cell.cell_data['base_segments']
    cell_type[cell.cell_data['metadata']['main_seg']['base']] = cell.get_ctype('manual')

In [38]:
segs_1 = set([a for b in base_segments[overlapping_cells[0]].values() for a in b])
segs_2 = set([a for b in base_segments[overlapping_cells[1]].values() for a in b])

print(f'{len(segs_1.difference(segs_2))} segments in cell 1 that are not in cell 2')
print(f'{len(segs_2.difference(segs_1))} segments in cell 2 that are not in cell 1')

4172 segments in cell 1 that are not in cell 2
2351 segments in cell 2 that are not in cell 1


In [39]:
overlap_seg_list = segs_1 & segs_2
print(f'{len(overlap_seg_list)} segments in both')

25 segments in both


set of cells from nodes

In [48]:

cell_color=['#33cc33','#cc33ff']
for i,n in enumerate(base_segments.keys()):
    with viewer.txn(overwrite=True) as s:
        color_structure = cell_color[i] # blue
        print(n)
        for bs in [a for b in base_segments[n].values() for a in b]:
           
            s.layers['base_segs'].segments.add(int(bs))
            s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

474236060
644824967


In [49]:
color_structure='#ff0000'

with viewer.txn(overwrite=True) as s:

    for bs in list(overlap_seg_list):
        s.layers['base_segs'].segments.add(int(bs))
        s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

## visualize output cell bodies

In [27]:
x = nodes[0]
len(base_segments[x]['unknown'])< len([a for b in base_segments[x].values() for a in b])

False

In [47]:
toplot=[]
for x in nodes:
    if cell_type[x] in ['lg','lf']:
        try:
            assert len(base_segments[x]['unknown']) < len([a for b in base_segments[x].values() for a in b])
            toplot.append(x)
        except: 
            continue
            # print(f'no cell structures labeled for {x} yet')

In [48]:
len(toplot)

16

In [56]:
cell_color={'lf':'#ff0000','lg':'#0000ff'}
for out_cell in toplot:
    with viewer.txn(overwrite=True) as s:
        color_structure = cell_color[cell_type[out_cell]] # blue

        for bs in random.choices(list(base_segments[out_cell]['unknown']),k=50):
            s.layers['base_segs'].segments.add(int(bs))
            s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

## MG-Output convergence

In [11]:
lf_innet = ['393063300'] #['307591597']
mg1_innet = ['214412684','300210608','387368998','472175645'] # ['389085521','213669553','472409584']
lg_innet = ['215526370']
mg2_innet = ['128473437','215572949','300316308','386117124','472051969','644825148']

In [13]:
cell_color = '#0000ff' # '#0000ff' # '#ff0000' #

for cell_id in mg1_innet:
    this_cell = base_segments[cell_id]
    all_segs = [a for b in this_cell.values() for a in b]

    with viewer.txn(overwrite=True) as s:
        for dtype in this_cell.keys():
            
            color_structure = cell_color # blue
            
            # if dtype=='axon':
            #     color_structure = "#ebecf0"
            
            for bs in this_cell[dtype]:
                s.layers['base_segs'].segments.add(int(bs))
                s.layers['base_segs'].segment_colors[int(bs)] = color_structure # blue

        # s.layers['base_segs'].segment_colors[int(self.cell_data['anchor_seg'])] = '#1e90ff'

In [14]:
download_path = Path('/Users/kperks/Downloads')
filepath = download_path / 'mg1-lf_x-y_.png'

s = viewer.screenshot()
with open(filepath, 'wb') as f:
    f.write(s.screenshot.image)

KeyboardInterrupt: 

### remove displayed segments

In [ ]:
displayed_segs = set([str(x) for x in viewer.state.layers['base_segs'].segments])

In [ ]:
# REMOVE SEGMENTS FROM NGviewer
remove_segs = displayed_segs

with viewer.txn(overwrite=True) as s:
    for bs in remove_segs:
        if int(bs) in s.layers['base_segs'].segments:
            s.layers['base_segs'].segments.remove(int(bs))